In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras.models import Model
%matplotlib inline

Using TensorFlow backend.


In [2]:
NUM_CLASSES = 4
model = Sequential()
model.add(ResNet50(include_top = False, 
                   weights=None, 
                   input_shape = (256, 256, 3), classes = NUM_CLASSES))

## Add in last 3 layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.load_weights('/Users/agunapal/Documents/berkeley/w210/model/weights-improvement-13-0.65.hdf5')